# Tensor Attributes

In [1]:
import torch

# Create a 2x2 tensor with float values and enable gradient tracking.
x = torch.tensor([[1, 2], [3, 4]], dtype=torch.float32, requires_grad=True)

print("Tensor x:")
print(x)


Tensor x:
tensor([[1., 2.],
        [3., 4.]], requires_grad=True)


### Inspect Core Tensor Attributes

In [2]:
# Core attributes of tensor x (a leaf tensor):
print("\n-- Core Attributes of x --")
print("dtype:         ", x.dtype)
print("device:        ", x.device)
print("shape:         ", x.shape)       # Same as x.size()
print("requires_grad: ", x.requires_grad)
print("grad:          ", x.grad)        # None until a backward pass is run.
print("grad_fn:       ", x.grad_fn)     # None because x is a leaf tensor.
print("layout:        ", x.layout)
print("is_leaf:       ", x.is_leaf)



-- Core Attributes of x --
dtype:          torch.float32
device:         cpu
shape:          torch.Size([2, 2])
requires_grad:  True
grad:           None
grad_fn:        None
layout:         torch.strided
is_leaf:        True


### Create a Non-Leaf Tensor and Check Its Attributes

In [3]:
# Perform an operation on x to create y. Now y is not a leaf tensor.
y = x + 2

print("\nTensor y (result of x + 2):")
print(y)

print("\n-- Attributes of Tensor y --")
print("dtype:         ", y.dtype)
print("device:        ", y.device)
print("shape:         ", y.shape)
print("requires_grad: ", y.requires_grad)
print("grad:          ", y.grad)        # Still None (no backward pass yet)
print("grad_fn:       ", y.grad_fn)     # Shows the function that created y.
print("layout:        ", y.layout)
print("is_leaf:       ", y.is_leaf)       # False – since it was created via an operation.



Tensor y (result of x + 2):
tensor([[3., 4.],
        [5., 6.]], grad_fn=<AddBackward0>)

-- Attributes of Tensor y --
dtype:          torch.float32
device:         cpu
shape:          torch.Size([2, 2])
requires_grad:  True
grad:           None
grad_fn:        <AddBackward0 object at 0x00000229B6AF2C20>
layout:         torch.strided
is_leaf:        False


C:\Users\aksha\AppData\Local\Temp\ipykernel_11096\562828982.py:12: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\build\aten\src\ATen/core/TensorBody.h:494.)
  print("grad:          ", y.grad)        # Still None (no backward pass yet)


### Demonstrate Additional Useful Attributes

In [4]:
# --- Additional Attributes ---

# Transpose: For a 2D tensor, x.T is a shorthand for x.transpose(0, 1)
print("\nTranspose of x (x.T):")
print(x.T)

# The .data attribute returns the underlying data (without gradient tracking).
print("\nUsing x.data (detached view):")
print(x.data)

# Check if the tensor is stored on a GPU (CUDA).
print("\nIs x on CUDA? ", x.is_cuda)  # Expected to be False on most machines unless using a GPU.

# MKL-DNN is a backend for CPU optimizations. Typically, default tensors are NOT MKL-DNN:
print("Is x MKL-DNN? ", x.is_mkldnn)



Transpose of x (x.T):
tensor([[1., 3.],
        [2., 4.]], grad_fn=<PermuteBackward0>)

Using x.data (detached view):
tensor([[1., 2.],
        [3., 4.]])

Is x on CUDA?  False
Is x MKL-DNN?  False


### Working with Quantized and Sparse Tensors

In [5]:
# --- Quantized Tensor Example ---
# To simulate quantization, create an integer tensor, convert to float, then quantize.
x_int = torch.randint(low=0, high=255, size=(3, 3), dtype=torch.uint8)
x_float = x_int.to(torch.float32)
# Quantize the tensor: scale and zero_point are hyperparameters.
x_quantized = torch.quantize_per_tensor(x_float, scale=0.1, zero_point=10, dtype=torch.quint8)

print("\nQuantized Tensor:")
print(x_quantized)
print("Is x_quantized quantized? ", x_quantized.is_quantized)


# --- Sparse Tensor Example ---
# Create a sparse tensor using sparse_coo_tensor.
indices = torch.tensor([[0, 1, 1],
                        [2, 0, 2]])
values = torch.tensor([3, 4, 5], dtype=torch.float32)
x_sparse = torch.sparse_coo_tensor(indices, values, size=(2, 3))

print("\nSparse Tensor:")
print(x_sparse)
print("Is x_sparse sparse? ", x_sparse.is_sparse)



Quantized Tensor:
tensor([[24.5000, 24.5000, 24.5000],
        [19.0000, 24.5000, 24.5000],
        [24.5000, 24.5000, 24.5000]], size=(3, 3), dtype=torch.quint8,
       quantization_scheme=torch.per_tensor_affine, scale=0.1, zero_point=10)
Is x_quantized quantized?  True

Sparse Tensor:
tensor(indices=tensor([[0, 1, 1],
                       [2, 0, 2]]),
       values=tensor([3., 4., 5.]),
       size=(2, 3), nnz=3, layout=torch.sparse_coo)
Is x_sparse sparse?  True


### Demonstrate Gradient Flow on a Non-Leaf Tensor

In [6]:
# Compute a simple loss by summing all elements in y.
loss = y.sum()
loss.backward()  # Performs backpropagation.

print("\nAfter backward pass:")
print("Gradient accumulated in x (x.grad):")
print(x.grad)      # Gradients are computed on x because of the operation that created y.
print("y.grad (non-leaf tensor):", y.grad)  # Remains None.



After backward pass:
Gradient accumulated in x (x.grad):
tensor([[1., 1.],
        [1., 1.]])
y.grad (non-leaf tensor): None


C:\Users\aksha\AppData\Local\Temp\ipykernel_11096\2227329156.py:8: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\build\aten\src\ATen/core/TensorBody.h:494.)
  print("y.grad (non-leaf tensor):", y.grad)  # Remains None.


### Additional Useful Methods

In [7]:
print("\n-- Additional Methods --")
print("x dimensions (x.dim()):", x.dim())
print("Total number of elements in x (x.numel()):", x.numel())
print("x strides: ", x.stride())



-- Additional Methods --
x dimensions (x.dim()): 2
Total number of elements in x (x.numel()): 4
x strides:  (2, 1)
